# R in Snowflake Workspace Notebooks

This notebook demonstrates how to use R within Snowflake Workspace Notebooks using rpy2.

**Capabilities:**
- Execute R code in `%%R` magic cells alongside Python
- Transfer data bidirectionally between Python and R
- Connect to Snowflake from R using ADBC (PAT or Key Pair) or Reticulate + Snowpark

**Sections:**
1. [Installation & Configuration](#section-1-installation--configuration)
2. [Python & R Interoperability](#section-2-python--r-interoperability)
3. [Snowflake via ADBC](#section-3-snowflake-database-connectivity) (PAT authentication)
4. [Key Pair Authentication](#section-4-alternative-authentication---key-pair-jwt) (alternative to PAT)
5. [Snowflake via Reticulate](#section-5-reticulate---access-snowpark-from-r) (no auth setup needed)
6. [Data Visualization with ggplot2](#section-6-data-visualization-with-ggplot2) (charts and plots)

---

# Section 1: Installation & Configuration

This section sets up R and rpy2 in the Workspace Notebook environment.

## Overview

Snowflake Workspace Notebooks run in containers with a managed Python kernel. To use R:

1. **Install R** via micromamba (lightweight conda-compatible package manager)
2. **Install rpy2** into the notebook's Python kernel
3. **Register `%%R` magic** for R cell support

## Customizing R Packages

Edit `r_packages.yaml` to customize which R packages are installed:

```yaml
# Conda-forge packages (installed via micromamba)
conda_packages:
  - r-base           # Required: Base R
  - r-tidyverse      # Data manipulation
  - r-yourpackage    # Add packages here

# CRAN packages (installed via install.packages)
cran_packages:
  - somepackage      # Packages not available on conda-forge
```

## Installation Options

| Command | Description |
|---------|-------------|
| `bash setup_r_environment.sh` | Basic R installation |
| `bash setup_r_environment.sh --adbc` | R + ADBC driver for Snowflake connectivity |
| `bash setup_r_environment.sh --verbose` | Show detailed logging |
| `bash setup_r_environment.sh --help` | Show all options |

### 1.1 Install R Environment

Run the setup script. Choose `--basic` for R only, or `--adbc` to include ADBC Snowflake driver.

**Note:** This step takes 2-5 minutes on first run. The `--adbc` option takes longer as it compiles the Snowflake driver.

The script includes:
- Pre-flight checks (disk space, network connectivity)
- Automatic retry for network operations
- Logging to `setup_r.log` for debugging

In [ ]:
# Choose ONE of the following:

# Option A: Basic R installation (faster)
# !bash setup_r_environment.sh --basic

# Option B: R + ADBC for Snowflake connectivity (required for Section 3)
!bash setup_r_environment.sh --adbc

### 1.2 Configure Python Environment & Install rpy2

This cell uses the helper module to:
1. Point Python to the R environment
2. Install rpy2 into the notebook kernel
3. Register the `%%R` magic
4. Load output helper functions for cleaner display

**Run this cell after the installation script completes.**

**Output Helpers:** Workspace Notebooks add extra line breaks to R output. After setup, use these R functions for cleaner formatting:

| Function | Usage | Description |
|----------|-------|-------------|
| `rprint(x)` | `rprint(df)` | Print any object cleanly |
| `rview(df, n)` | `rview(iris, n=10)` | View data frame with optional row limit |
| `rglimpse(df)` | `rglimpse(df)` | Glimpse data frame structure |

In [ ]:
# Method 1: Using the helper module (recommended)
import sys
sys.path.insert(0, '.')  # Ensure current directory is in path

from r_helpers import setup_r_environment

result = setup_r_environment()

if result['success']:
    print("✓ R environment configured successfully")
    print(f"  R version: {result['r_version']}")
    print(f"  rpy2 installed: {result['rpy2_installed']}")
    print(f"  %%R magic registered: {result['magic_registered']}")
else:
    print("✗ Setup failed:")
    for error in result['errors']:
        print(f"  - {error}")

In [ ]:
# Method 2: Manual configuration (if helper module not available)
# Uncomment and run if Method 1 fails

# import os
# import sys
# import subprocess

# ENV_PREFIX = "/root/.local/share/mamba/envs/r_env"
# os.environ["PATH"] = f"{ENV_PREFIX}/bin:" + os.environ["PATH"]
# os.environ["R_HOME"] = f"{ENV_PREFIX}/lib/R"

# subprocess.run([sys.executable, "-m", "pip", "install", "rpy2", "-q"], check=True)

# from rpy2.ipython import rmagic
# get_ipython().register_magics(rmagic.RMagics)
# print("R environment configured")

### 1.3 Verify R Installation

Test that R is working correctly.

In [ ]:
%%R
# Print R version (simple output works fine)
R.version.string

In [ ]:
%%R
# List installed packages
# Use rprint() for cleaner output in Workspace Notebooks
ip <- as.data.frame(installed.packages()[, c(1, 3:4)])
ip <- ip[is.na(ip$Priority), 1:2, drop = FALSE]
rprint(ip)

### 1.4 Run Diagnostics (Optional)

Run comprehensive environment diagnostics to verify all components are working.

In [ ]:
from r_helpers import check_environment, print_diagnostics

# Run and display diagnostics
print_diagnostics()

---

# Section 2: Python & R Interoperability

This section demonstrates how to work with data in both Python and R, including:
- Using the `%%R` magic for R cells
- Passing data from Python to R
- Passing data from R to Python
- Running R functions from Python

## 2.1 Using %%R Magic Cells

The `%%R` magic lets you write R code directly in a cell. The magic supports flags:

| Flag | Description |
|------|-------------|
| `-i var` | Import Python variable `var` into R |
| `-o var` | Export R variable `var` back to Python |
| `-w WIDTH` | Set plot width |
| `-h HEIGHT` | Set plot height |

In [ ]:
%%R
# Basic R operations
x <- c(1, 2, 3, 4, 5)
mean(x)

In [ ]:
%%R
# Using tidyverse
library(dplyr)

rprint(
data.frame(
  name = c("Alice", "Bob", "Charlie"),
  score = c(85, 92, 78)
) %>%
  mutate(grade = case_when(
    score >= 90 ~ "A",
    score >= 80 ~ "B",
    TRUE ~ "C"
  ))
)  

## 2.2 Passing Data: Python → R

Use the `-i` flag to pass Python objects into R cells.

In [ ]:
# Create a pandas DataFrame in Python
import pandas as pd

python_df = pd.DataFrame({
    'city': ['New York', 'Los Angeles', 'Chicago', 'Houston'],
    'population': [8336817, 3979576, 2693976, 2320268],
    'area_sq_mi': [302.6, 468.7, 227.3, 670.6]
})

print("Python DataFrame:")
python_df

In [ ]:
%%R -i python_df
# The Python DataFrame is now available in R as 'python_df'
library(dplyr)

cat("Received DataFrame in R:\n")
rglimpse(python_df)  # Use rglimpse() for clean output

# Perform R operations
result <- python_df %>%
  mutate(density = population / area_sq_mi) %>%
  arrange(desc(density))

rprint(result)  # Use rprint() for clean output

## 2.3 Passing Data: R → Python

Use the `-o` flag to export R objects back to Python.

In [ ]:
%%R -o r_result
# Create a data frame in R
r_result <- data.frame(
  x = 1:10,
  y = (1:10)^2,
  label = paste0("Point_", 1:10)
)

cat("Created R data.frame:\n")
rprint(r_result)  # Use rprint() for clean output

In [ ]:
# The R data.frame is now available in Python
print("R result in Python:")
print(type(r_result))
print(r_result)

## 2.4 Using R from Python (without magic)

For more control, you can use rpy2's Python API directly.

In [ ]:
import rpy2.robjects as ro
from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri
from rpy2.robjects.conversion import localconverter

# Import R packages
base = importr('base')
stats = importr('stats')

# Run R code and get results
result = ro.r('sum(1:100)')
print(f"Sum of 1 to 100: {result[0]}")

In [ ]:
# Convert pandas DataFrame to R and run R functions on it
import pandas as pd
import rpy2.robjects as ro
from rpy2.robjects import pandas2ri

# Create sample data
df = pd.DataFrame({
    'x': [1, 2, 3, 4, 5],
    'y': [2.1, 3.9, 6.2, 7.8, 10.1]
})

# Convert to R and run linear regression
with (ro.default_converter + pandas2ri.converter).context():
    r_df = ro.conversion.get_conversion().py2rpy(df)

# Run linear regression in R
lm_result = stats.lm('y ~ x', data=r_df)
print("Linear Regression Results:")
print(base.summary(lm_result))

## 2.5 Working with R's Built-in Datasets

Access R's built-in datasets and convert them to pandas.

In [ ]:
import rpy2.robjects as ro
from rpy2.robjects import pandas2ri
from rpy2.robjects.conversion import localconverter

# Load the iris dataset in R
ro.r("data(iris)")

# Get the R data.frame
iris_r = ro.r["iris"]

# Convert to pandas DataFrame
with localconverter(ro.default_converter + pandas2ri.converter):
    iris_df = pandas2ri.rpy2py(iris_r)

print("Iris dataset (first 10 rows):")
iris_df.head(10)

---

# Section 3: Snowflake Database Connectivity

This section demonstrates connecting to Snowflake from R using ADBC.

**Prerequisites:**
- Run the setup script with `--adbc` flag (Section 1.1)
- Have appropriate Snowflake permissions

## Authentication Options

| Method | Status | Notes |
|--------|--------|-------|
| Python `get_active_session()` | ✅ Works | Use for Snowpark queries, bridge to R via rpy2 |
| ADBC with PAT | ✅ Works | Direct R-to-Snowflake, requires PAT token |
| SPCS OAuth Token | ❌ Blocked | Container token not authorized for ADBC |
| Username/Password | ❌ Blocked | SPCS requires OAuth |

## Connection Management

This notebook uses connection pooling - the ADBC connection is stored as `r_sf_con` in R's global environment and reused across cells. This avoids the overhead of creating new connections for each query.

## 3.1 Setup Python Session

First, establish the standard Python Snowpark session.

In [ ]:
from snowflake.snowpark.context import get_active_session

session = get_active_session()

# Print connection details
print("Snowflake Connection:")
print(f"  Account:   {session.sql('SELECT CURRENT_ACCOUNT()').collect()[0][0]}")
print(f"  User:      {session.sql('SELECT CURRENT_USER()').collect()[0][0]}")
print(f"  Role:      {session.get_current_role()}")
print(f"  Database:  {session.get_current_database()}")
print(f"  Schema:    {session.get_current_schema()}")
print(f"  Warehouse: {session.get_current_warehouse()}")

## 3.2 Create Programmatic Access Token (PAT)

ADBC requires a PAT for authentication. Use the `PATManager` helper class for token management.

**Features:**
- Automatic expiry tracking
- Token refresh when needed
- Secure environment variable storage

**Security Notes:**
- PAT is stored in environment variable (not persisted)
- Token expires after specified duration
- Use `ROLE_RESTRICTION` to limit token scope

In [ ]:
from r_helpers import PATManager

# Initialize PAT manager
pat_mgr = PATManager(session)

# Create a new PAT (or refresh existing one)
result = pat_mgr.create_pat(
    days_to_expiry=1,           # Token valid for 1 day
    force_recreate=True,        # Replace existing token
    network_policy_bypass_mins=240  # 4 hour bypass
)

if result['success']:
    print(f"✓ PAT created successfully")
    print(f"  User: {result['user']}")
    print(f"  Role restriction: {result['role_restriction']}")
    print(f"  Expires: {result['expires_at']}")
else:
    print(f"✗ PAT creation failed: {result['error']}")

In [ ]:
# Check PAT status at any time
status = pat_mgr.get_status()
print("PAT Status:")
for key, value in status.items():
    print(f"  {key}: {value}")

## 3.3 Validate ADBC Prerequisites

Before connecting, validate that all ADBC prerequisites are met.

In [ ]:
from r_helpers import validate_adbc_connection

valid, message = validate_adbc_connection()
print(message)

## 3.4 Initialize R Connection Management

Load the connection management functions into R. This provides:
- `get_snowflake_connection()` - Get or create connection (stored as `r_sf_con`)
- `close_snowflake_connection()` - Close and release connection
- `is_snowflake_connected()` - Check connection status
- `snowflake_connection_status()` - Get detailed status

In [ ]:
from r_helpers import init_r_connection_management

success, msg = init_r_connection_management()
print(msg)

## 3.5 Connect to Snowflake from R (ADBC)

Use `get_snowflake_connection()` to establish or reuse the ADBC connection.

The connection is stored as `r_sf_con` in R's global environment and is automatically reused in subsequent cells.

In [ ]:
%%R
# Get or create the Snowflake connection
# Connection is stored globally as r_sf_con
r_sf_con <- get_snowflake_connection()

# Show connection status (uses print_connection_status() for clean output)
print_connection_status()

## 3.6 Query Snowflake from R

Run queries using the `r_sf_con` connection. The connection is automatically reused across cells.

In [ ]:
%%R
# Simple test query using r_sf_con
r_sf_con |>
  read_adbc("SELECT CURRENT_USER() AS USER, CURRENT_ROLE() AS ROLE, CURRENT_WAREHOUSE() AS WAREHOUSE") |>
  tibble::as_tibble()

In [ ]:
%%R
# Query sample data from Snowflake
# Using the shared SNOWFLAKE_SAMPLE_DATA database
nations <- r_sf_con |>
  read_adbc("
    SELECT N_NATIONKEY, N_NAME, N_REGIONKEY 
    FROM SNOWFLAKE_SAMPLE_DATA.TPCH_SF1.NATION 
    ORDER BY N_NATIONKEY
    LIMIT 10
  ") |>
  tibble::as_tibble()

nations

In [ ]:
%%R
# More complex query with aggregation
library(dplyr)

orders_summary <- r_sf_con |>
  read_adbc("
    SELECT 
      O_ORDERSTATUS,
      COUNT(*) as ORDER_COUNT,
      SUM(O_TOTALPRICE) as TOTAL_VALUE,
      AVG(O_TOTALPRICE) as AVG_VALUE
    FROM SNOWFLAKE_SAMPLE_DATA.TPCH_SF1.ORDERS
    GROUP BY O_ORDERSTATUS
    ORDER BY ORDER_COUNT DESC
  ") |>
  tibble::as_tibble()

orders_summary

In [ ]:
%%R
# Verify connection is being reused (not recreated)
cat("Connection still valid:", is_snowflake_connected(), "\n")

## 3.7 Query from Python, Analyze in R

An alternative pattern: use Python's Snowpark session for querying, then pass data to R for analysis.

In [ ]:
# Query using Python Snowpark session
customers_df = session.sql("""
    SELECT 
        C_CUSTKEY,
        C_NAME,
        C_NATIONKEY,
        C_ACCTBAL
    FROM SNOWFLAKE_SAMPLE_DATA.TPCH_SF1.CUSTOMER
    LIMIT 100
""").to_pandas()

print(f"Retrieved {len(customers_df)} rows")
customers_df.head()

In [ ]:
%%R -i customers_df
# Analyze the data in R
library(dplyr)

cat("Summary Statistics for Customer Account Balance:\n")
rprint(summary(customers_df$C_ACCTBAL))

cat("\nCustomers by Nation (top 5):\n")
result <- customers_df %>%
  group_by(C_NATIONKEY) %>%
  summarise(
    count = n(),
    avg_balance = mean(C_ACCTBAL),
    total_balance = sum(C_ACCTBAL)
  ) %>%
  arrange(desc(count)) %>%
  head(5)

rprint(result)  # Use rprint() for clean output

## 3.8 Check Connection Status

You can check the connection status from either Python or R.

In [ ]:
# Check status from Python
from r_helpers import get_r_connection_status

status = get_r_connection_status()
print("R Connection Status (from Python):")
for key, value in status.items():
    print(f"  {key}: {value}")

In [ ]:
%%R
# Get or create the Snowflake connection
# Connection is stored globally as r_sf_con
r_sf_con <- get_snowflake_connection()

# Show connection status (uses print_connection_status() for clean output)
print_connection_status()

## 3.9 Clean Up

Close ADBC connection and optionally remove the PAT.

In [ ]:
%%R
# Close the Snowflake connection
close_snowflake_connection()

In [ ]:
# Alternative: Close from Python
# from r_helpers import close_r_connection
# success, msg = close_r_connection()
# print(msg)

In [ ]:
# Remove PAT when done (optional)
# pat_mgr.remove_pat()
# print("PAT removed")

---

# Section 4: Alternative Authentication - Key Pair (JWT)

This section demonstrates Key Pair (JWT) authentication as an alternative to PAT.

## Authentication Methods for R ADBC

| Method | Status | Notes |
|--------|--------|-------|
| **PAT (Programmatic Access Token)** | ✅ Working | **Recommended** - easiest to set up (see Section 3) |
| **Key Pair (JWT)** | ✅ Working | Alternative - no token expiry, shown below |
| SPCS OAuth Token | ❌ Blocked | Container token restricted to specific connectors |
| Username/Password | ❌ Blocked | SPCS enforces OAuth for internal connections |

> **Note:** For tests of non-working methods, see `archive/auth_methods_not_working.ipynb`

## Prerequisites

- ADBC installed (`--adbc` flag during setup)
- RSA key pair generated
- Public key registered with your Snowflake user

## 4.1 Load Alternative Auth Test Functions

Load the R functions for testing different authentication methods.

In [ ]:
from r_helpers import init_r_alt_auth

success, msg = init_r_alt_auth()
print(msg)

## 4.2 Key Pair (JWT) Authentication ✅

Key pair authentication uses RSA keys instead of passwords. **This method is confirmed working** and is a good alternative to PAT for users who prefer key-based auth.

**Setup steps:**
1. Generate an RSA key pair
2. Register the public key with your Snowflake user
3. Use the private key for authentication

### Step 1: Generate Key Pair (Python)

In [ ]:
from r_helpers import KeyPairAuth

# Initialize key pair auth helper
kp_auth = KeyPairAuth()

# Generate a new key pair (or use load_private_key() for existing key)
# Note: Requires 'cryptography' package: pip install cryptography
result = kp_auth.generate_key_pair(
    key_size=2048,
    output_dir="/tmp",
    passphrase=None  # Set a passphrase for encrypted key
)

if result['success']:
    print("✓ Key pair generated successfully")
    print(f"  Private key: {result['private_key_path']}")
    print(f"  Public key:  {result['public_key_path']}")
    print(f"\n  Public key for Snowflake registration:")
    print(f"  {result['public_key_for_snowflake'][:50]}...")
else:
    print(f"✗ Key generation failed: {result['error']}")

### Step 2: Register Public Key with Snowflake

Run this SQL to register the public key with your user (requires ACCOUNTADMIN or appropriate privileges).

In [ ]:
# Generate SQL for registering the public key
if result['success']:
    sql = kp_auth.register_public_key_sql(result['public_key_for_snowflake'])
    print("Run this SQL to register the public key:")
    print("-" * 60)
    print(sql)
    print("-" * 60)
    print("\nOr run via Snowpark session:")
    print("  session.sql(sql).collect()")

In [ ]:
session.sql(sql).collect()

### Step 3: Configure and Test Key Pair Auth

In [ ]:
# Configure environment for key pair auth
config = kp_auth.configure_for_adbc()
print("Key Pair Auth Configuration:")
for key, value in config.items():
    print(f"  {key}: {value}")

In [ ]:
%%R
# Test key pair authentication
# Note: Public key must be registered with user first!
result <- test_keypair_auth()
rprint(result)

## 4.3 Authentication Summary

### Working Methods

| Method | Auth Type | Best For |
|--------|-----------|----------|
| **PAT** | `auth_pat` | Most use cases - easy programmatic setup |
| **Key Pair** | `auth_jwt` | Long-lived credentials without expiry |

### Non-Working Methods (Blocked by SPCS)

| Method | Reason |
|--------|--------|
| SPCS OAuth Token | Restricted to specific Snowflake connectors |
| Username/Password | SPCS enforces OAuth internally |

> See `archive/auth_methods_not_working.ipynb` for test code if needed.

---

# Section 5: Reticulate - Access Snowpark from R

This section demonstrates using **reticulate** to access the Python Snowpark session directly from R. This is an alternative to ADBC that leverages the notebook's built-in authentication.

## Advantages of Reticulate Approach

| Feature | Reticulate + Snowpark | ADBC |
|---------|----------------------|------|
| Authentication | Uses notebook's built-in auth | Requires PAT or Key Pair |
| Setup | No additional auth setup | PAT creation or key registration |
| Connection | Shares Python session | Separate R connection |
| Best for | Quick queries, prototyping | Production R pipelines |

## How It Works

1. R accesses Python's Snowpark session via reticulate
2. Execute SQL queries using `session$sql()`
3. Convert results to pandas DataFrame with `.to_pandas()`
4. Reticulate automatically converts pandas → R data.frame

## Output Pattern

For best display in Notebooks, use `%%R -o variable` to export R data frames to Python, then display them in a subsequent Python cell. This lets the Notebook render the DataFrame with proper formatting.

## 5.1 Setup Reticulate

Configure reticulate to use the notebook's Python environment.

> **Note:** You may see a warning about reticulate/rpy2 compatibility. This is safe to ignore if using reticulate >= 1.25 (installed by default). The issue was fixed in reticulate PR #1188.

In [ ]:
%%R
library(reticulate)

# Use the same Python that's running the notebook kernel
# This ensures we access the same Snowpark session
use_python(Sys.which("python3"), required = TRUE)

# Verify Python is accessible
py_config()

## 5.2 Access Snowpark Session from R

Import the Snowpark module and get the active session. This uses the notebook's built-in authentication - no PAT required!

In [ ]:
%%R
# Import Snowpark module
snowpark <- import("snowflake.snowpark")

# Get the active session (uses notebook's built-in auth)
session <- snowpark$Session$builder$getOrCreate()

# Verify connection
rcat("Connected to Snowflake via Snowpark!")
rcat("Account: ", session$get_current_account())
rcat("User: ", session$get_current_user())
rcat("Database: ", session$get_current_database())
rcat("Schema: ", session$get_current_schema())

## 5.3 Query Snowflake and Get R DataFrame

Execute SQL queries and convert results to R data frames.

**Output Pattern:** Use `%%R -o variable` to export results to Python, then display in the next cell for nice Notebook formatting.

In [ ]:
%%R -o nations_df
# Execute a query and get Snowpark DataFrame
# Use -o to export result to Python for nice display
nations_df <- session$sql("
    SELECT N_NATIONKEY, N_NAME, N_REGIONKEY 
    FROM SNOWFLAKE_SAMPLE_DATA.TPCH_SF1.NATION 
    LIMIT 10
")$to_pandas()

# Print data type (R sees this as a data.frame)
cat("R data type:", class(nations_df), "\n")

In [ ]:
# Display the exported DataFrame (nice Notebook rendering)
nations_df

## 5.4 R Analysis on Snowflake Data

Perform R analysis using dplyr on data retrieved via Snowpark. Use `-o` to export the result for display.

In [ ]:
%%R -o customer_analysis
# Query customer data with aggregation
customers_df <- session$sql("
    SELECT 
        C_MKTSEGMENT,
        COUNT(*) as CUSTOMER_COUNT,
        AVG(C_ACCTBAL) as AVG_BALANCE,
        MIN(C_ACCTBAL) as MIN_BALANCE,
        MAX(C_ACCTBAL) as MAX_BALANCE
    FROM SNOWFLAKE_SAMPLE_DATA.TPCH_SF1.CUSTOMER
    GROUP BY C_MKTSEGMENT
    ORDER BY AVG_BALANCE DESC
")$to_pandas()

# Use dplyr for additional analysis
library(dplyr)

customer_analysis <- customers_df %>%
    mutate(
        BALANCE_RANGE = MAX_BALANCE - MIN_BALANCE,
        SEGMENT_SIZE = case_when(
            CUSTOMER_COUNT > 30000 ~ "Large",
            CUSTOMER_COUNT > 29000 ~ "Medium",
            TRUE ~ "Small"
        )
    )

cat("Analysis complete - result exported to Python\n")

In [ ]:
# Display the R analysis result (exported via -o)
customer_analysis

## 5.5 Helper Function for Snowpark Queries

Create a convenience function to simplify querying. Use `-o` to export results for display.

In [ ]:
%%R -o orders_summary
#' Query Snowflake via Snowpark and return R data.frame
#' 
#' @param sql SQL query string
#' @return R data.frame with query results
snowpark_query <- function(sql) {
    session$sql(sql)$to_pandas()
}

# Example usage - export result with -o
orders_summary <- snowpark_query("
    SELECT 
        O_ORDERSTATUS,
        COUNT(*) as ORDER_COUNT,
        SUM(O_TOTALPRICE) as TOTAL_VALUE
    FROM SNOWFLAKE_SAMPLE_DATA.TPCH_SF1.ORDERS
    GROUP BY O_ORDERSTATUS
")

cat("Query complete - orders_summary exported to Python\n")

In [ ]:
# Display the orders summary (exported via -o)
orders_summary

## 5.6 Reticulate vs ADBC Comparison

| Aspect | Reticulate + Snowpark | ADBC (Section 3 & 4) |
|--------|----------------------|----------------------|
| **Authentication** | Automatic (notebook's session) | PAT or Key Pair required |
| **Setup complexity** | Minimal | Moderate |
| **Data path** | Snowflake → Snowpark → pandas → R | Snowflake → Arrow → R |
| **Performance** | Good for moderate data | Better for large data (Arrow) |
| **R-native** | No (via Python) | Yes (native R driver) |
| **Best for** | Quick analysis, prototyping | Production R workflows |

### When to Use Each

**Use Reticulate + Snowpark when:**
- You need quick access without auth setup
- Working interactively/prototyping
- Data sizes are moderate (< 1M rows)
- You're already using Python and R together

**Use ADBC when:**
- Building production R pipelines
- Working with large datasets
- Need pure R solution
- Require connection pooling/management

---

# Section 6: Data Visualization with ggplot2

This section demonstrates creating visualizations with **ggplot2** and displaying them in the Notebook.

## Key Points

- ggplot2 is included via `tidyverse` (installed by default)
- Use `%%R -w WIDTH -h HEIGHT` to control plot dimensions (in pixels)
- Call `print(p)` explicitly to render the plot
- Plots render inline in the Notebook output

## Plot Size Parameters

| Parameter | Description | Example |
|-----------|-------------|---------|
| `-w` | Width in pixels | `-w 800` |
| `-h` | Height in pixels | `-h 500` |
| `--type` | Graphics device | `--type=cairo` (optional, better quality) |

## 6.1 Basic ggplot2 Example

Create a simple scatter plot using the built-in `mtcars` dataset.

In [ ]:
%%R -w 700 -h 450
library(ggplot2)

# Basic scatter plot with mtcars
p <- ggplot(mtcars, aes(x = wt, y = mpg, color = factor(cyl))) +
    geom_point(size = 3) +
    labs(
        title = "Fuel Efficiency vs Weight",
        x = "Weight (1000 lbs)",
        y = "Miles per Gallon",
        color = "Cylinders"
    ) +
    theme_minimal()

print(p)

## 6.2 Visualize Snowflake Data

Query Snowflake data and create a bar chart. Bar charts work best when values have meaningful differences from zero.

> **Tip:** Avoid bar charts when values are clustered in a narrow range (e.g., all ~$4,500). Use dot plots or adjust the visualization instead.

In [ ]:
%%R -w 800 -h 500
library(ggplot2)
library(dplyr)

# Query Snowflake for order data by status
# This data has more variance for a meaningful bar chart
orders <- session$sql("
    SELECT 
        O_ORDERSTATUS,
        COUNT(*) as ORDER_COUNT,
        ROUND(SUM(O_TOTALPRICE) / 1e9, 2) as TOTAL_VALUE_BILLIONS
    FROM SNOWFLAKE_SAMPLE_DATA.TPCH_SF1.ORDERS
    GROUP BY O_ORDERSTATUS
    ORDER BY TOTAL_VALUE_BILLIONS DESC
")$to_pandas()

# Create bar chart - good when values have meaningful differences
p <- ggplot(orders, aes(x = reorder(O_ORDERSTATUS, -TOTAL_VALUE_BILLIONS), 
                         y = TOTAL_VALUE_BILLIONS)) +
    geom_col(aes(fill = ORDER_COUNT), width = 0.6) +
    geom_text(aes(label = paste0("$", TOTAL_VALUE_BILLIONS, "B")), 
              vjust = -0.5, size = 4) +
    scale_fill_viridis_c(option = "plasma", labels = scales::comma) +
    scale_y_continuous(expand = expansion(mult = c(0, 0.1))) +
    labs(
        title = "Total Order Value by Status",
        subtitle = "Data from Snowflake TPC-H Sample",
        x = "Order Status",
        y = "Total Value ($ Billions)",
        fill = "Order\nCount"
    ) +
    theme_minimal(base_size = 12) +
    theme(
        plot.title = element_text(face = "bold")
    )

print(p)

## 6.3 Multi-Panel Visualization (Facets)

Create faceted plots to compare distributions across categories.

In [ ]:
%%R -w 900 -h 600
library(ggplot2)
library(dplyr)

# Query order data by status and priority
orders <- session$sql("
    SELECT 
        O_ORDERSTATUS,
        O_ORDERPRIORITY,
        O_TOTALPRICE
    FROM SNOWFLAKE_SAMPLE_DATA.TPCH_SF1.ORDERS
    LIMIT 5000
")$to_pandas()

# Create faceted histogram
p <- ggplot(orders, aes(x = O_TOTALPRICE, fill = O_ORDERSTATUS)) +
    geom_histogram(bins = 30, alpha = 0.7) +
    facet_wrap(~O_ORDERPRIORITY, scales = "free_y", ncol = 3) +
    scale_x_continuous(labels = scales::dollar_format(scale = 0.001, suffix = "K")) +
    scale_fill_brewer(palette = "Set2") +
    labs(
        title = "Order Value Distribution by Priority",
        subtitle = "Colored by Order Status",
        x = "Total Price",
        y = "Count",
        fill = "Status"
    ) +
    theme_light(base_size = 11) +
    theme(
        plot.title = element_text(face = "bold"),
        strip.text = element_text(face = "bold")
    )

print(p)

## 6.4 Saving and Loading Plots

Use `ggsave()` to export plots to files, then display them from Python using `IPython.display.Image`.

In [ ]:
%%R -w 700 -h 450
library(ggplot2)

# Create a plot
p <- ggplot(mtcars, aes(x = hp, y = mpg)) +
    geom_point(aes(color = factor(gear)), size = 3) +
    geom_smooth(method = "lm", se = TRUE, color = "darkgray") +
    labs(
        title = "MPG vs Horsepower",
        x = "Horsepower",
        y = "Miles per Gallon",
        color = "Gears"
    ) +
    theme_bw()

# Display the plot inline
print(p)

# Save to file
ggsave("/tmp/mpg_vs_hp.png", p, width = 8, height = 5, dpi = 150)
cat("Plot saved to /tmp/mpg_vs_hp.png\n")

In [ ]:
# Display the saved PNG file in the notebook
from IPython.display import Image, display

display(Image(filename="/tmp/mpg_vs_hp.png"))

---

## Troubleshooting

### Common Issues

| Issue | Solution |
|-------|----------|
| `ModuleNotFoundError: No module named 'rpy2'` | Run Section 1.2 to install rpy2 |
| `R.version.string` returns error | Verify PATH and R_HOME are set correctly |
| ADBC `auth_pat` error | Ensure PAT was created and stored in `SNOWFLAKE_PAT` |
| Network policy error | PAT may need `MINS_TO_BYPASS_NETWORK_POLICY_REQUIREMENT` |
| `adbcsnowflake` not found | Ensure setup script ran with `--adbc` flag |
| Setup script fails | Check `setup_r.log` for detailed error messages |
| `r_sf_con` not found | Run `get_snowflake_connection()` to create connection |

### Run Full Diagnostics

In [ ]:
# Comprehensive diagnostic check
from r_helpers import print_diagnostics
print_diagnostics()

In [ ]:
# Quick environment check
import os
import shutil

print("Quick Environment Check:")
print(f"  R_HOME: {os.environ.get('R_HOME', 'NOT SET')}")
print(f"  R binary: {shutil.which('R') or 'NOT FOUND'}")
print(f"  SNOWFLAKE_ACCOUNT: {os.environ.get('SNOWFLAKE_ACCOUNT', 'NOT SET')}")
print(f"  SNOWFLAKE_PAT: {'SET' if os.environ.get('SNOWFLAKE_PAT') else 'NOT SET'}")

In [ ]:
# View setup log if something went wrong
# !tail -50 setup_r.log